<a href="https://colab.research.google.com/github/ritwikraha/computer-needs-glasses/blob/master/image-object-tracking/Object_Tracking_for_UFC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Step 1: Set up the Colab environment
!pip install -q ultralytics
!pip install -q opencv-python-headless
!pip install -q moviepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.4/779.4 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 62.8 MB/s eta 0:00:00


In [2]:
from ultralytics import YOLO, SAM
import cv2
import moviepy.editor as mpy
from google.colab import files
import numpy as np


In [3]:
# Upload the video file
uploaded = files.upload()

# Define the video file path
video_path = list(uploaded.keys())[0]

Saving gathjevpoirier2.mp4 to gathjevpoirier2.mp4


In [4]:
# Initialize the YOLOv8 model for object detection
detection_model = YOLO("yolov8n.pt")  # You can use any YOLOv8 model

# Initialize the SAM model for segmentation
sam_model = SAM("sam_b.pt")

# Initialize the YOLOv8 model for pose detection
pose_model = YOLO("yolov8n-pose.pt")

100%|██████████| 6.23M/6.23M [00:00<00:00, 370MB/s]


100%|██████████| 358M/358M [00:00<00:00, 453MB/s]


100%|██████████| 6.51M/6.51M [00:00<00:00, 368MB/s]


In [10]:
# Open the video file
cap = cv2.VideoCapture(video_path)
# Get the video writer initialized to save the output video
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

In [11]:
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret:
        # Step 2: Detect bounding boxes of persons
        detection_results = detection_model(frame)

        # Extract bounding boxes of the first two detected persons
        person_bboxes = []
        person_count = 0
        for result in detection_results:
            for i, bbox in enumerate(result.boxes.xyxy):
                if result.names[int(result.boxes.cls[i])] == "person":
                    person_bboxes.append(bbox.cpu().numpy().tolist())
                    person_count += 1
                    if person_count >= 2:
                        break
            if person_count >= 2:
                break

        if person_bboxes:
            annotated_frame = frame.copy()
            for person_bbox in person_bboxes:
                # Step 3: Segment each person using SAM model with the detected bounding box
                x1, y1, x2, y2 = map(int, person_bbox)
                mask_results = sam_model(frame, bboxes=[[x1, y1, x2, y2]])

                # Debug: print structure of mask_results
                print("mask_results structure:", mask_results)

                # Extract the mask
                mask = mask_results[0].masks.data[0].cpu().numpy()

                # Apply mask to the frame
                segmented_frame = cv2.bitwise_and(frame, frame, mask=mask.astype(np.uint8))

                # Step 4: Run pose detection on the segmented person
                pose_results = pose_model(segmented_frame)

                # Extract and draw poses on the black background frame
                for result in pose_results:
                    pose_annotated_frame = result.plot()
                    mask_inv = cv2.bitwise_not(mask.astype(np.uint8))
                    frame_bg = cv2.bitwise_and(annotated_frame, annotated_frame, mask=mask_inv)
                    person_fg = cv2.bitwise_and(pose_annotated_frame, pose_annotated_frame, mask=mask.astype(np.uint8))
                    annotated_frame = cv2.add(frame_bg, person_fg)

            # Write the frame into the output video
            out.write(annotated_frame)
        else:
            # If no person is detected, write the original frame
            out.write(frame)
    else:
        break

# Release the video capture and writer objects
cap.release()
out.release()

Streaming output truncated to the last 5000 lines.
Speed: 2.2ms preprocess, 8.7ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 1024x1024 243.8ms
Speed: 6.3ms preprocess, 243.8ms inference, 0.2ms postprocess per image at shape (1, 3, 1024, 1024)
mask_results structure: [ultralytics.engine.results.Results object with attributes:

boxes: None
keypoints: None
masks: ultralytics.engine.results.Masks object
names: {0: '0'}
obb: None
orig_img: array([[[134,  54,  54],
        [133,  53,  53],
        [133,  53,  53],
        ...,
        [ 24,   1,   9],
        [ 24,   1,   9],
        [ 24,   1,   9]],

       [[140,  60,  60],
        [139,  59,  59],
        [139,  59,  59],
        ...,
        [ 24,   1,   9],
        [ 24,   1,   9],
        [ 24,   1,   9]],

       [[139,  65,  64],
        [139,  65,  64],
        [139,  65,  64],
        ...,
        [ 28,   0,   9],
        [ 28,   0,   9],
        [ 28,   0,   9]],

       ...,

       [[229, 191, 167],
  

In [12]:
# Step 5: Display the processed video
output_video_path = 'output.avi'

# Convert the video to MP4 format to display in Colab
clip = mpy.VideoFileClip(output_video_path)
clip.write_videofile("output-ufc-generated.mp4")

Moviepy - Building video output-ufc-generated.mp4.
Moviepy - Writing video output-ufc-generated.mp4



Moviepy - Done !
Moviepy - video ready output-ufc-generated.mp4


In [13]:
from google.colab import files

# Replace 'your_file.txt' with the actual filename you want to download
filename = "output-ufc-generated.mp4"

# Download the file
files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>